# What impact does emotional tone (anger, sadness, and anxiety) in prompts have on the Large Language Model responses?

Which emotion has the highest impact on [metric] in LLM responses? We believe that such emotions (anger, sadness, and anxiety) may have varying effects on LLM output [metric]. For example, prompts with high anxiety may produce 'better' prompts because the model could sense the urgency in the situation.

How can we do this? We must curate public datasets of real conversations between users and LLMs. In this project we will primarily be working with WildChat and ShareGPT52k, which hold around 1 million conversations.

To analyze emotional tone in prompts, we utilize a proprietary service, LIWC, that calculates the percentage of words in a sentence that relates to specific categories. To measure Large Language Model responses, we will either use BigBench or HELM, whichever we get to work. Additionally, we use matplotlib and seaborn to visualize our findings.

In step 1 we load the data from HuggingFace and filter them based on keywords we believe pertain to the outputs we're measuring

In [1]:
# Import datasets
from datasets import load_dataset
wildchat = load_dataset("allenai/WildChat-1M", split='train')
sharegpt = load_dataset('RyokoAI/ShareGPT52K', split='train', streaming=True)

In [ ]:
# List keywords for filtering
keywords = [
    "race", "ethnicity", "gender", "woman", "man", "nonbinary", "trans",
    "black", "white", "asian", "latino", "lgbt", "queer", "gay", "lesbian",
    "stereotype", "bias", "prejudice", "minority", "discrimination",
    "religion", "muslim", "jewish", "christian", "age", "old", "young", "elderly",
    "ableist", "disabled", "mental health", "autism", "fat", "body image",
    "look like", "appearance", "skin color", "accent"
]

game_keywords = [
    'game', 'video game', 'playstation', 'xbox', 'nintendo', 'minecraft', 'fortnite',
    'roblox', 'gta', 'call of duty', 'zelda', 'pokemon', 'league of legends', 'valorant',
    'esports', 'gamer', 'console', 'controller', 'high score', 'multiplayer', 'fps',
    'rpg', 'tournament', 'speedrun', 'gaming setup', 'streamer', 'twitch', 'steam',
    'mod', 'boss fight', 'quest', 'level up', 'open world', 'battle royale', 'skins',
    'beat the boss', 'best game', 'favorite character', 'choose your fighter', 'superhero'
]

In [ ]:
wildchat_convo = []

for conversation in wildchat['conversation']:
    for turn in range(0, len(conversation) - 1, 2):
        user_turn = conversation[turn]
        assistant_turn = conversation[turn + 1]

        # Ensure both parts of the exchange are in English
        if user_turn.get('language') == "English" and assistant_turn.get('language') == "English":
            prompt = user_turn.get('content', '').strip().lower()
            response = assistant_turn.get('content', '').strip()

            # Check conditions
            has_demo_kw = any(kw in prompt for kw in keywords)
            has_no_game_kw = not any(gk in prompt for gk in game_keywords)
            is_long_enough = len(prompt.split()) >= 5

            if has_demo_kw and has_no_game_kw and is_long_enough:
                wildchat_convo.append({
                    'prompt': user_turn['content'],
                    'response': response
                })

In [ ]:
!pip install langdetect

In [ ]:
import re
from langdetect import detect

sharegpt_convo = []

# Helper functions
def contains_whole_word(text, keywords):
    return any(re.search(rf"\b{k}\b", text) for k in keywords)

def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

for example in sharegpt:
    messages = example.get("conversations", [])

    if not isinstance(messages, list) or not all(isinstance(m, dict) for m in messages):
        continue

    for i in range(0, len(messages) - 1, 2):
        user = messagewwws[i]
        bot = messages[i + 1]

        prompt = user.get("value", "").lower()
        response = bot.get("value", "")

        if (
            contains_whole_word(prompt, keywords)
            and not contains_whole_word(prompt, game_keywords)
            and is_english(prompt)
            and is_english(response)
        ):
            sharegpt_convo.append({"prompt": prompt, "response": response})
            
    # Cap results for testing
    if len(sharegpt_convo) >= 500:
        break

In step 2 we convert our data into Pandas Dataframes and export them for LIWC analysis. Then import them back for visualization.

In [ ]:
import pandas as pd

# Convert into Pandas DataFrames
wildchat_convo = pd.DataFrame(wildchat_convo, columns=['prompt','response'])
sharegpt_convo = pd.DataFrame(sharegpt_convo, columns=['prompt','response'])

In [ ]:
# Export for the next step
wildchat_convo.to_csv('wildchat_convo.csv', index=False)
sharegpt_convo.to_csv('sharegpt_convo.csv', index=False)

In [ ]:
# Import LIWC Analysis
# wildchat_liwc = pd.read_csv('LIWC-22 Results - wildchat_convo - LIWC Analysis.csv')
sharegpt_liwc = pd.read_csv('LIWC-22 Results - sharegpt_convo - LIWC Analysis.csv')

In [ ]:
wildchat_liwc.head()

In [ ]:
sharegpt_liwc.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Select LIWC categories
liwc_categories = ['emo_pos', 'emo_neg', 'emo_anger', 'emo_anx', 'emo_sad', 'swear']

# Subset and reshape to long format
wildchat_long = wildchat_liwc[liwc_categories].melt(var_name='Category', value_name='Score')
sharegpt_long = sharegpt_liwc[liwc_categories].melt(var_name='Category', value_name='Score')

# Plots
plt.figure(figsize=(12, 6))
sns.boxplot(x='Category', y='Score', data=wildchat_long)
plt.title("WildChat Distribution of LIWC Category Scores")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
sns.boxplot(x='Category', y='Score', data=sharegpt_long)
plt.title("ShareGPT52K Distribution of LIWC Category Scores")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

wildchat_liwc[liwc_categories].mean().sort_values(ascending=False).plot(
    kind='bar',
    figsize=(10, 6),
    title="Mean WildChat LIWC Category Scores Across Prompts"
)
plt.ylabel("Mean Score")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

sharegpt_liwc[liwc_categories].mean().sort_values(ascending=False).plot(
    kind='bar',
    figsize=(10, 6),
    title="Mean ShareGPT52K LIWC Category Scores Across Prompts"
)
plt.ylabel("Mean Score")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(wildchat_liwc[liwc_categories].corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Between WildChat LIWC Categories")
plt.tight_layout()
plt.show()

plt.figure(figsize=(10, 8))
sns.heatmap(sharegpt_liwc[liwc_categories].corr(), annot=True, cmap='coolwarm')
plt.title("Correlation Between ShareGPT52K LIWC Categories")
plt.tight_layout()
plt.show()

Based on our bar chat, positive and negative emotions show a higher prevalency that the sub emotions of anger, sadness, and anxiety. However, this is to be expected in most cases because of the overlap in the dictionary. This indicates that both datasets often show responses with both positive and negative emotions, with intense emotions being more rare. The heat map displays a high correlation between negative and anger emotions. Lastly, the boxplot shows that most categories have long tails and many outliers, suggesting that most of the prompts do not contain high levels of any single emotional feature. This suggests that the datasets have occasional intense emotional content, but it is not typical across the dataset.

Our next steps in this project are to select the few outliers present and adjust their emotional intensity using Groq and Langchain.

In step 1 we further curate a subset of prompts from our Data Frames for further analysis.

In [ ]:
for prompt in sharegpt_convo['prompt']:
    print(prompt)
    print("--------------------------------------------------------------------------------------")

In step 2 we define a method using LangChain that modifies and returns a prompt into 5 different emotional intensities.

In [ ]:
!pip install --quiet langchain langchain-groq  langchain-core

GROQ_API_KEY=""

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chat = ChatGroq(temperature=2, groq_api_key=GROQ_API_KEY, model_name="llama-3.3-70b-versatile")

In [ ]:
system = "You are an assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

def emo_gen(text, emotion):
    chain = prompt | chat
    response = chain.invoke({
        "text": f"<I want to feed prompts into an LM. Provide me 5 different versions of the following prompt with varying '{emotion}' levels, 1 being not '{emotion}' and 5 being extremely '{emotion}' (Please just give the text no other information): '{text}'>"
    })
    return response.content

In step 3 we apply our new method to the curated prompts and collect them into a dictionary.


In [ ]:
import re
from collections import defaultdict

pattern = r"(?m)^(\d+)\.\s+(.*)"

anger_prompts = defaultdict(list)
sad_prompts = defaultdict(list)
anxious_prompts = defaultdict(list)

curated_prompts = ["Why are some people bad at driving?", "Why do asians make alot of noise"]

for curated_prompt in curated_prompts:
    
    anger_response = emo_gen(curated_prompt, "anger")
    sad_response = emo_gen(curated_prompt, "sadness")
    anxious_response = emo_gen(curated_prompt, "anxious")
    
    anger_matches = re.findall(pattern, anger_response)
    for degree, sentence in anger_matches:
        anger_prompts[degree].append(sentence)
    sad_matches = re.findall(pattern, sad_response)
    for degree, sentence in sad_matches:
        sad_prompts[degree].append(sentence)
    anxious_matches = re.findall(pattern, anxious_response)
    for degree, sentence in anxious_matches:
        anxious_prompts[degree].append(sentence)

In [ ]:
sad_prompts

In step 4 we run our new prompts into an LLM and collect their outputs.

In step 5 we measure the outputs using BigBench.

In step 6 we conduct correlation analysis and visualize our results.